In [1]:
import sys
sys.path.insert(0, '../../../network')

In [2]:
import logging
import argparse
import numpy as np
from network import Population, RateNetwork
from learning import ReachingTask
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse, ThresholdPlasticityRule
from sequences import GaussianSequence
import matplotlib.pyplot as plt
# np.set_printoptions(threshold=sys.maxsize)
logging.basicConfig(level=logging.INFO)

In [3]:
phi = ErrorFunction(mu=0.22, sigma=0.1).phi
ctx = Population(N=3000, tau=1e-2, phi=phi, name='ctx')
bg = Population(N=3000, tau=1e-2, phi=phi, name='bg')
plasticity = ThresholdPlasticityRule(x_f=0.5, q_f=0.8) 

S, P = 1, 4
sequences_ctx = [GaussianSequence(P,ctx.size, seed=11) for i in range(S)]
patterns_ctx = np.stack([s.inputs for s in sequences_ctx])
sequences_bg = [GaussianSequence(P,ctx.size, seed=367) for i in range(S)]
patterns_bg = np.stack([s.inputs for s in sequences_bg])

J_cc = SparseConnectivity(source=ctx, target=ctx, p=0.5)
synapse_cc = LinearSynapse(J_cc.K, A=5)
J_cc.store_attractors(patterns_ctx[0], patterns_ctx[0], synapse_cc.h_EE, 
                      plasticity.f, plasticity.g)
J_bb = SparseConnectivity(source=bg, target=bg, p=0.5)
synapse_bb = LinearSynapse(J_bb.K, A=5)
J_bb.store_attractors(patterns_bg[0], patterns_bg[0], synapse_bb.h_EE, 
                      plasticity.f, plasticity.g)
J_cb  = SparseConnectivity(source=bg, target=ctx, p=0.5)
synapse_cb = LinearSynapse(J_cb.K, A=3.5)
J_cb.store_attractors(patterns_bg[0], patterns_ctx[0], synapse_cb.h_EE, 
                      plasticity.f, plasticity.g)

J_bc = SparseConnectivity(source=ctx, target=bg, p=0.5)
synapse_bc = LinearSynapse(J_bc.K, A=0)
J_bc.store_sequences(patterns_ctx, patterns_bg, synapse_bc.h_EE, plasticity.f, plasticity.g)
J_bc.update_sequences(patterns_ctx[0][1], patterns_bg[0][2],
                   3.5, lamb=1,f=plasticity.f, g=plasticity.g)
net_ctx = RateNetwork(ctx, c_EE=J_cc, c_IE=J_bc, formulation=4)
net_bg = RateNetwork(bg, c_II=J_bb, c_EI=J_cb, formulation=4)

INFO:connectivity:Building connections from ctx to ctx
INFO:connectivity:Storing attractors
100%|█████████████████████████████████████| 3000/3000 [00:01<00:00, 2707.32it/s]
INFO:connectivity:Building connections from bg to bg
INFO:connectivity:Storing attractors
100%|█████████████████████████████████████| 3000/3000 [00:01<00:00, 2705.64it/s]
INFO:connectivity:Building connections from bg to ctx
INFO:connectivity:Storing attractors
100%|█████████████████████████████████████| 3000/3000 [00:00<00:00, 3519.99it/s]
INFO:connectivity:Building connections from ctx to bg
INFO:connectivity:Storing sequences
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix


In [4]:
init_input_ctx = np.random.RandomState().normal(0,1,size=patterns_ctx[0][0].shape)
init_input_bg = np.random.RandomState().normal(0,1,size=patterns_bg[0][0].shape)
# init_input_ctx = patterns_ctx[0][2]
# init_input_bg = patterns_bg[0][2]
T=10
mouse = ReachingTask(3, alpha=0.8)
net_ctx.simulate_learning(mouse, net_bg, T, init_input_ctx, init_input_bg, 
                                      patterns_ctx[0], patterns_bg[0], plasticity, noise=25)

INFO:network:Integrating network dynamics
  1%|▍                                       | 107/9999 [00:20<25:55,  6.36it/s]

lick-->reach out 0 0


  9%|███▌                                    | 879/9999 [02:20<24:21,  6.24it/s]

reach-->retract out 0 0


 10%|████                                   | 1043/9999 [02:45<23:36,  6.32it/s]

...

 99%|██████████████████████████████████████▊| 9938/9999 [31:15<00:59,  1.02it/s]

lick-->reach in 0 0.3178197978436753


100%|██████████████████████████████████████▉| 9989/9999 [31:28<00:09,  1.02it/s]

reach-->retract in 1 0.27851247382014005


100%|███████████████████████████████████████| 9999/9999 [31:30<00:00,  5.29it/s]


In [5]:
init_input_ctx = np.random.RandomState().normal(0,1,size=patterns_ctx[0][0].shape)
init_input_bg = np.random.RandomState().normal(0,1,size=patterns_bg[0][0].shape)
T=1
net_ctx.simulate_euler2(net_bg, T, init_input_ctx, init_input_bg, noise=25)

INFO:network:Integrating network dynamics
100%|█████████████████████████████████████████| 999/999 [02:39<00:00,  6.25it/s]


In [6]:
overlaps = sequences_ctx[0].overlaps(net_ctx, ctx, phi=phi)
correlations = sequences_ctx[0].overlaps(net_ctx, ctx, phi=phi, correlation=True)

INFO:sequences:Computing overlaps
100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.36it/s]
INFO:sequences:Computing correlations
100%|█████████████████████████████████████████████| 4/4 [00:38<00:00,  9.53s/it]


In [8]:
np.savez('/Users/stanleypark/Desktop/code/hebbian_sequence_learning/figures/article/1/output/behaviors1.npz', 
         behaviors=mouse.behaviors, correlations=correlations)